In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import *
!pip install torchaudio librosa pydub ffmpeg
clear_output()

In [ ]:
# model
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output
import torchaudio
import torch
import torch.nn as nn
from sklearn import model_selection
from sklearn import metrics
from tabulate import tabulate
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import warnings
warnings.filterwarnings("ignore")
class AudioLSTM(nn.Module):

    def __init__(self, n_feature=5, out_feature=1, n_hidden=256, n_layers=2, drop_prob=0.5):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_feature = n_feature

        self.lstm = nn.LSTM(self.n_feature, self.n_hidden, self.n_layers, dropout=self.drop_prob, batch_first=True)

        self.dropout = nn.Dropout(drop_prob)

        self.fc = nn.Linear(n_hidden, out_feature)

    def forward(self, x, hidden):
        # x.shape (batch, seq_len, n_features)
        l_out, l_hidden = self.lstm(x, hidden)

        # out.shape (batch, seq_len, n_hidden*direction)
        out = self.dropout(l_out)

        # out.shape (batch, out_feature)
        out = self.fc(out[:, -1, :])

        # return the final output and the hidden state
        return out, l_hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data

        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        return hidden

In [ ]:
import IPython.display as ipd
import torchaudio
SOUND_PATH = "/content/drive/MyDrive/DREAM/Wakewords/input/cv-other-dev"
class AudioDataset:
  def __init__(self, sound_path, class_id, augmentations=None):
    self.sound_path = sound_path
    self.class_id = class_id
    self.augmentations = augmentations
    
  def __len__(self):
    return len(self.sound_path)

  def __getitem__(self, idx):
    path = self.sound_path[idx]
    waveform, sr = torchaudio.load(path)
    audio_mono = torch.mean(waveform, dim=0, keepdim=True) 
    tempData = torch.zeros([1, 160000])
    if audio_mono.numel() < 160000:
        tempData[:, :audio_mono.numel()] = audio_mono
    else:
        tempData = audio_mono[:, :160000]
    audio_mono=tempData
    mel_specgram = torchaudio.transforms.MelSpectrogram(sr)(audio_mono)
    mel_specgram_norm = (mel_specgram - mel_specgram.mean()) / mel_specgram.std()
    mfcc = torchaudio.transforms.MFCC(sample_rate=sr)(audio_mono)
#         print(f'mfcc {mfcc.size()}')
    mfcc_norm = (mfcc - mfcc.mean()) / mfcc.std()
    new_feat = torch.cat([mel_specgram, mfcc], axis=1)
    
    return {
        "specgram": torch.tensor(new_feat[0].permute(1, 0), dtype=torch.float),
        "label": torch.tensor(self.class_id[idx], dtype=torch.float)
    }

In [ ]:
# Import All Necessary Packages

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output
import torchaudio
import torch
import torch.nn as nn
from sklearn import model_selection
from sklearn import metrics
from sklearn import metrics
from tabulate import tabulate
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_fn(data):
    specs = []
    labels = []
    for d in data:
        spec = d["specgram"].to(device)
        label = d["label"].to(device)
        specs.append(spec)
        labels.append(label)
    spec = torch.nn.utils.rnn.pad_sequence(specs, batch_first=True, padding_value=0.)
    labels = torch.tensor(labels)
    return spec, labels


# FILE_PATH = "/content/drive/MyDrive/DREAM/Wakewords/input/cv-other-dev"

if __name__ == "__main__":
    df = pd.read_csv("/content/drive/MyDrive/DREAM/Wakewords/input/example250.csv")
    files = df["files"].values.tolist()
    # folder_fold = df["fold"].values
    label = df["label"].values.tolist()
    path = [
        os.path.join(SOUND_PATH, file) for file in files
    ]
    
    X_train, X_test, y_train, y_test = model_selection.train_test_split(path, label, random_state=42, test_size=0.3)
    
    train_dataset = AudioDataset(
        sound_path=X_train,
        class_id=y_train
    )
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=8, shuffle=True, drop_last=True, collate_fn=collate_fn
    )
    
    test_dataset = AudioDataset(
        sound_path=X_test,
        class_id=y_test
    )
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=8, shuffle=False, drop_last=True, collate_fn=collate_fn
    )

In [ ]:
from sklearn import metrics
from tabulate import tabulate
from tqdm import tqdm

def save_model(model, filename):
  torch.save(model, filename)
  print("-> Model Saved")


def binary_accuracy(preds, y):
  rounded_preds = torch.round(preds)
  acc = rounded_preds.eq(y.view_as(rounded_preds)).sum().item() / len(y)
  return acc

def train(epoch, data_loader, model, optimizer, device):
  losses = []
  preds = []
  labels = []
  model.train()
  loop = tqdm(data_loader)
  for idx, (spec, label) in enumerate(loop):
    spec, label = spec.to(device), label.to(device)
    optimizer.zero_grad()
    output, hidden_state = model(spec, model.init_hidden(8))
    output = torch.flatten(output)
    # print(label.view(-1, 1).unsqueeze(1).shape)
    loss = nn.BCEWithLogitsLoss()(output, label)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    pred = torch.sigmoid(output)
    preds += torch.flatten(torch.round(pred)).cpu()
    labels += torch.flatten(label).cpu()
    loop.set_description(f"EPOCH: {epoch} | ITERATION : {idx}/{len(data_loader)} | LOSS: {loss.item()}")
    loop.set_postfix(loss=loss.item())
  avg_train_loss = sum(losses) / len(losses)
  acc = binary_accuracy(torch.tensor(preds), torch.tensor(labels))
  print(f"Avg. train loss: {avg_train_loss}")
  report = metrics.classification_report(torch.tensor(labels).numpy(), torch.tensor(preds).numpy())
  print(report)

  return acc, report

def evaluate(data_loader, model, optimizer, device):
  accs = []
  preds = []
  labels = []
  loop = tqdm(data_loader)
  with torch.no_grad():
    for idx, (mfcc, label) in enumerate(loop):
      mfcc, label = mfcc.to(device), label.to(device)
      output, hidden_state = model(mfcc, model.init_hidden(8))
      pred = torch.sigmoid(output)
      acc = binary_accuracy(pred, label)
      preds += torch.flatten(torch.round(pred)).cpu()
      labels += torch.flatten(label).cpu()
      accs.append(acc)
      print("Iter: {}/{}, accuracy: {}".format(idx, len(test_loader), acc), end="\r")
  average_acc = sum(accs)/len(accs) 
  print('Average test Accuracy:', average_acc, "\n")
  report = metrics.classification_report(labels, preds)
  print(report)
  return average_acc, report


def main():
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = AudioLSTM(n_feature=168, out_feature=1).to(device)
  optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
  best_acc = 0
  best_epoch = 0
  for epoch in range(20):
    train_acc, train_report = train(epoch, train_loader, model, optimizer, device)
    test_acc, test_report = evaluate(test_loader, model, optimizer, device)
    if train_acc > best_acc:
      best_acc = train_acc
      best_epoch = epoch
      filename = f"model_epoch_{epoch}.pth.tar"
      checkpoint = {"model":AudioLSTM(n_feature=168, out_feature=1), "state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
      save_model(checkpoint, filename)
    table = [
            ["train_acc", train_acc], ["test_acc", test_acc],
            ["best_acc", best_acc], ["best_epoch", epoch]
          ]
    print(tabulate(table))
main()

EPOCH: 0 | ITERATION : 39/40 | LOSS: 0.6436424255371094: 100%|██████████| 40/40 [02:58<00:00,  4.46s/it, loss=0.644]
  0%|          | 0/17 [00:00<?, ?it/s]

Avg. train loss: 0.6787327796220779
              precision    recall  f1-score   support

         0.0       0.69      0.48      0.56       166
         1.0       0.58      0.77      0.66       154

    accuracy                           0.62       320
   macro avg       0.64      0.62      0.61       320
weighted avg       0.64      0.62      0.61       320



  6%|▌         | 1/17 [00:03<00:58,  3.64s/it]

 12%|█▏        | 2/17 [00:06<00:52,  3.52s/it]

 18%|█▊        | 3/17 [00:11<00:52,  3.76s/it]

...

 82%|████████▏ | 14/17 [00:03<00:00,  3.78it/s]

 88%|████████▊ | 15/17 [00:04<00:00,  3.72it/s]

 94%|█████████▍| 16/17 [00:04<00:00,  3.38it/s]

100%|██████████| 17/17 [00:04<00:00,  3.65it/s]

Average test Accuracy: 0.9852941176470589 

              precision    recall  f1-score   support

         0.0       1.00      0.97      0.99        80
         1.0       0.97      1.00      0.98        56

    accuracy                           0.99       136
   macro avg       0.98      0.99      0.98       136
weighted avg       0.99      0.99      0.99       136

----------  ---------
train_acc    1
test_acc     0.985294
best_acc     1
best_epoch  19
----------  ---------


In [ ]:
torch.load("/content/model_epoch_4.pth.tar")

In [ ]:
waveform, sr = torchaudio.load("/content/drive/MyDrive/DREAM/Wakewords/input/cv-other-dev/sample-000181.mp3")
audio_mono = torch.mean(waveform, dim=0, keepdim=True)
tempData = torch.zeros([1, 160000])
if audio_mono.numel() < 160000:
    tempData[:, :audio_mono.numel()] = audio_mono
else:
    tempData = audio_mono[:, :160000]
audio_mono=tempData
mel_specgram = torchaudio.transforms.MelSpectrogram(sr)(audio_mono)
mel_specgram_norm = (mel_specgram - mel_specgram.mean()) / mel_specgram.std()
mfcc = torchaudio.transforms.MFCC(sample_rate=sr)(audio_mono)
#         print(f'mfcc {mfcc.size()}')
mfcc_norm = (mfcc - mfcc.mean()) / mfcc.std()
new_feat = torch.cat([mel_specgram, mfcc], axis=1)

In [ ]:
waveform, sr = torchaudio.load("/content/drive/MyDrive/DREAM/Wakewords/input/wake/0 - Copy (3).wav")
audio_mono = torch.mean(waveform, dim=0, keepdim=True)
tempData = torch.zeros([1, 160000])
if audio_mono.numel() < 160000:
    tempData[:, :audio_mono.numel()] = audio_mono
else:
    tempData = audio_mono[:, :160000]
audio_mono=tempData
mel_specgram = torchaudio.transforms.MelSpectrogram(sr)(audio_mono)
mel_specgram_norm = (mel_specgram - mel_specgram.mean()) / mel_specgram.std()
mfcc = torchaudio.transforms.MFCC(sample_rate=sr)(audio_mono)
#         print(f'mfcc {mfcc.size()}')
mfcc_norm = (mfcc - mfcc.mean()) / mfcc.std()
new_feat = torch.cat([mel_specgram, mfcc], axis=1)

data = torch.utils.data.DataLoader(new_feat.permute(0, 2, 1))
model = torch.load("/content/model_epoch_4.pth.tar", map_location=torch.device("cpu"))["model"]
new = torch.load("/content/model_epoch_4.pth.tar", map_location=torch.device("cpu"))["state_dict"]

model.load_state_dict(new)
model.eval().cpu()
with torch.no_grad():
  for x in data:
    x = x.to("cpu")
    output, hidden_state = model(x, (torch.zeros(2, 1, 256), torch.zeros(2, 1, 256)))
    print(torch.round(torch.sigmoid(output)))

tensor([[1.]])


In [ ]:
model.init_hidden(1)[1]

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
(torch.zeros(2, 1, 256), torch.zeros(2, 1, 256))

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [ ]:
!pip install torchaudio

     |████████████████████████████████| 1.9MB 6.7MB/s 


In [ ]:
import torchaudio
torchaudio.__version__

'0.8.1'

In [ ]:
import torch
torch.__version__

'1.8.1+cu101'

In [ ]:
# model
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output
import torchaudio
import torch
import torch.nn as nn
from sklearn import model_selection
from sklearn import metrics
from tabulate import tabulate
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import warnings
warnings.filterwarnings("ignore")
class AudioLSTM(nn.Module):

    def __init__(self, n_feature=5, out_feature=1, n_hidden=256, n_layers=2, drop_prob=0.5):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_feature = n_feature

        self.lstm = nn.LSTM(self.n_feature, self.n_hidden, self.n_layers, dropout=self.drop_prob, batch_first=True)

        self.dropout = nn.Dropout(drop_prob)

        self.fc = nn.Linear(n_hidden, out_feature)

    def forward(self, x, hidden):
        # x.shape (batch, seq_len, n_features)
        l_out, l_hidden = self.lstm(x, hidden)

        # out.shape (batch, seq_len, n_hidden*direction)
        out = self.dropout(l_out)

        # out.shape (batch, out_feature)
        out = self.fc(out[:, -1, :])

        # return the final output and the hidden state
        return out, l_hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data

        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        return hidden

In [ ]:
# model = AudioLSTM(n_feature=168, out_feature=1)
a = torch.load("/content/model_epoch_4.pth.tar", map_location=torch.device("cpu"))["model"]
a

AudioLSTM(
  (lstm): LSTM(168, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)